In [12]:
#  Copyright (c) 2020.
#  Alireza Ghasemieh
#  a.ghasemieh65@gmail.com
#  https://github.com/ghasemieh

from Bugzilla_API import API_data_extract
from text_processing import preprocessing
import pandas as pd
import sqlite3

# Extract data from Bugzilla website
data_df = API_data_extract('2h')

In [34]:
# Create db if not exist
def create_table():
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS bug_report_table (summary TEXT,	component TEXT,	"
               "status TEXT, id INTEGER, creation_time TEXT, product TEXT, processed_summary TEXT )")
    connection.commit()
    connection.close()

def insert(id,product,component,creation_time,summary,processed_summary,status):
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("INSERT INTO bug_report_table VALUES(?,?,?,?,?,?,?)",
                   (id,product,component,creation_time,summary,processed_summary,status))
    connection.commit()
    connection.close()

def view():
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM bug_report_table")
    rows = cursor.fetchall()
    connection.close()
    return rows

def delete(id):
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("DELETE FROM bug_report_table WHERE id=?",(id,))
    connection.commit()
    connection.close()

In [ ]:
# Preprocess the data_df
data_list = []
for tup in data_df.itertuples():
    processed_summary = preprocessing(data_df,tup.id,'summary')
    data_list.append([tup.id,tup.product,tup.component,tup.creation_time,tup.summary,processed_summary,tup.status])
new_data_df = pd.DataFrame(data_list,columns = ["id","product","component","creation_time","summary","processed_summary","status"])


In [ ]:
# Save into a SQL database
create_table()
for tup in new_data_df.itertuples():
    insert(tup.id,tup.product,tup.component,tup.creation_time,tup.summary,tup.processed_summary,tup.status)


In [ ]:
view()

In [ ]:
delete(1610190)

In [ ]:
view()

In [15]:
# Save into a SQL database
create_table()
for tup in new_data_df.itertuples():
    insert(tup.id,tup.product,tup.component,tup.creation_time,tup.summary,tup.processed_summary,tup.status)


In [21]:
view()

[('1610190',
  'Thunderbird',
  'Preferences',
  '2020-01-19T17:38:01Z',
  'Wrong location of Thunderbird profile directory on MacOS',
  'wrong location thunderbird profile director',
  'UNCONFIRMED'),
 ('1610191',
  'WebExtensions',
  'General',
  '2020-01-19T18:09:41Z',
  'WebExtensions Find API - highlightResults not working anymore since FF71',
  'web extension find api highlight result',
  'UNCONFIRMED'),
 ('1610193',
  'Firefox',
  'Untriaged',
  '2020-01-19T18:40:31Z',
  'Memory increases with each reload on youtube',
  'memory increase reload youtube',
  'UNCONFIRMED'),
 ('1610194',
  'Firefox for Android',
  'Testing',
  '2020-01-19T19:04:59Z',
  'geckodriver cant start session on Firefox preview',
  'gecko driver start session firefox preview',
  'UNCONFIRMED'),
 ('1610195',
  'Firefox for iOS',
  'General',
  '2020-01-19T19:11:58Z',
  'iOS auto correct/predict dictionary is updated even in Private Browsing mode',
  'ios auto correct predict dictionary update',
  'UNCONFIRMED

In [39]:
delete(1610190)

In [40]:
view()

[('1610190',
  'Thunderbird',
  'Preferences',
  '2020-01-19T17:38:01Z',
  'Wrong location of Thunderbird profile directory on MacOS',
  'wrong location thunderbird profile director',
  'UNCONFIRMED'),
 ('1610191',
  'WebExtensions',
  'General',
  '2020-01-19T18:09:41Z',
  'WebExtensions Find API - highlightResults not working anymore since FF71',
  'web extension find api highlight result',
  'UNCONFIRMED'),
 ('1610193',
  'Firefox',
  'Untriaged',
  '2020-01-19T18:40:31Z',
  'Memory increases with each reload on youtube',
  'memory increase reload youtube',
  'UNCONFIRMED'),
 ('1610194',
  'Firefox for Android',
  'Testing',
  '2020-01-19T19:04:59Z',
  'geckodriver cant start session on Firefox preview',
  'gecko driver start session firefox preview',
  'UNCONFIRMED'),
 ('1610195',
  'Firefox for iOS',
  'General',
  '2020-01-19T19:11:58Z',
  'iOS auto correct/predict dictionary is updated even in Private Browsing mode',
  'ios auto correct predict dictionary update',
  'UNCONFIRMED